In [2]:
import pyspark

import findspark

findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark_session = SparkSession.builder.master("local[*]").appName('rdds').getOrCreate()

23/03/02 17:54:37 WARN Utils: Your hostname, idowu-pc resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlo1)
23/03/02 17:54:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 17:54:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark_session.read.parquet("data/pq/green/*/*")

```
 SELECT 
    -- Reveneue grouping 
   
    date_trunc('hour', pickup_datetime) as hour,
     PULocationID as revenue_zone,

    -- Revenue calculation 
    sum(total_amount) as amount,
    COUNT(1) AS number_records
    FROM green_trip_data
    WHERE pickup_datetime >= '2020-01-01 00:00'
    GROUP BY 
        1,2
```

In [5]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       1| 2020-01-24 22:00:56|  2020-01-24 22:06:12|                 N|         1|          95|         196|              1|         null|        6.0|  0.5|    0.

In [31]:
green_rdd = df_green \
    .select('lpep_pickup_datetime','total_amount','PULocationID')\
    .rdd

In [14]:
from datetime import datetime

In [17]:
start = datetime(year=2021, month=1, day=1)

In [19]:
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [21]:
rows = green_rdd.take(10)
row = rows[0]

In [30]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 22, 0, 56), total_amount=7.3)

In [34]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    amount = row.total_amount
    count = 1
    value = (amount, count)
    
    return (key, value)

In [37]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    output_value = (output_amount, output_count)
    
    return (output_amount, output_count)

In [52]:
from collections import namedtuple

In [53]:
RevenueRow = namedtuple('RevenueRow', ['hour','zone','revenue','count'])

In [59]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)])

In [54]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1], 
        revenue=row[1][0], 
        count=row[1][1])

In [60]:
df_result = green_rdd \
          .filter(lambda row: filter_outliers) \
          .map(prepare_for_grouping) \
          .reduceByKey(calculate_revenue) \
          .map(unwrap) \
          .toDF(result_schema)

In [63]:
df_result.write.parquet('data/tmp/green-revenue')

23/03/02 20:55:28 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [58]:
from pyspark.sql import types

In [7]:
df_green.take(5)

[Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 22, 0, 56), lpep_dropoff_datetime=datetime.datetime(2020, 1, 24, 22, 6, 12), store_and_fwd_flag='N', RatecodeID=1, PULocationID=95, DOLocationID=196, passenger_count=1, trip_distance=None, fare_amount=6.0, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=7.3, payment_type=2, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 14, 46), lpep_dropoff_datetime=datetime.datetime(2020, 1, 16, 15, 1), store_and_fwd_flag=None, RatecodeID=None, PULocationID=130, DOLocationID=10, passenger_count=None, trip_distance=None, fare_amount=21.57, extra=2.75, mta_tax=0.0, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=24.62, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 13, 7, 58, 32), lpep

Spark MapPartition

In [64]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID','trip_distance']

In [66]:
duration_rdd = df_green.select(columns).rdd

In [101]:
df_green.select(columns).schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_distance', IntegerType(), True)])

In [76]:
import pandas as pd

In [77]:
rows = duration_rdd.take(10)

In [79]:
pd.DataFrame(rows, columns=columns)

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,1.0,2020-01-24 22:00:56,95,196,None
1,NaN,2020-01-16 14:46:00,130,10,None
2,2.0,2020-01-13 07:58:32,166,142,None
3,NaN,2020-01-09 12:57:00,97,256,None
4,1.0,2020-01-08 13:45:35,116,116,None
5,2.0,2020-01-05 11:25:46,25,52,None
6,1.0,2020-01-24 08:38:07,42,74,None
7,NaN,2020-01-26 11:26:00,41,41,None
8,2.0,2020-01-16 09:46:53,226,157,None
9,2.0,2020-01-28 18:37:13,97,228,None


In [86]:
def model_predict(df):
    y_pred = df.trip_distance * 5
    return y_pred

In [130]:
def apply_model_in_batch(partition):
    df = pd.DataFrame(partition, columns= columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples(index=False):
        yield row

In [134]:
schem_model = types.StructType([
                            types.StructField('VendorID', types.DoubleType(), True), 
                          types.StructField('lpep_pickup_datetime', types.TimestampType(), True), 
                          types.StructField('PULocationID', types.IntegerType(), True), types.StructField('DOLocationID', types.IntegerType(), True),
                          types.StructField('trip_distance', types.DoubleType(), True),
                          types.StructField('predicted_duration',types.DoubleType(), True)])

In [136]:
duration_rdd.mapPartitions(apply_model_in_batch).take(5)

[Pandas(VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-24 22:00:56'), PULocationID=95, DOLocationID=196, trip_distance=None, predicted_duration=nan),
 Pandas(VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-16 14:46:00'), PULocationID=130, DOLocationID=10, trip_distance=None, predicted_duration=nan),
 Pandas(VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-13 07:58:32'), PULocationID=166, DOLocationID=142, trip_distance=None, predicted_duration=nan),
 Pandas(VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-09 12:57:00'), PULocationID=97, DOLocationID=256, trip_distance=None, predicted_duration=nan),
 Pandas(VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-08 13:45:35'), PULocationID=116, DOLocationID=116, trip_distance=None, predicted_duration=nan)]

In [67]:
duration_rdd.take(5)

[Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 22, 0, 56), PULocationID=95, DOLocationID=196, trip_distance=None),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 14, 46), PULocationID=130, DOLocationID=10, trip_distance=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 13, 7, 58, 32), PULocationID=166, DOLocationID=142, trip_distance=None),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 9, 12, 57), PULocationID=97, DOLocationID=256, trip_distance=None),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 8, 13, 45, 35), PULocationID=116, DOLocationID=116, trip_distance=None)]